# Parsing data from twitter

- Getting counts within time ranges and saving them into file time_ranges.pkl.
- Creating a df_all dataframe, that contains rows labeled with timeperiod based on tweet count thresholds.
- Collecting tweets within detected time ranges and saving into file balenciaga_nov.pkl.
- Retreiving information on users whose tweets are collected into file users_balenciaga2_nov_additional.pkl.
- Merging tweets and infirmation on users and saving into file balenciaga3.pkl.

# Libraries to import

In [ ]:
import tweepy
import json
import pandas as pd
from tqdm.auto import tqdm
import time
import requests

# Twitter tokens
Hided tokens for security measures.

In [1]:
at = ''
ats = ''
bt = ''

api_key = ''
api_secret_key = ''
access_token = ''
access_token_secret = ''

# Scarping time ranges for tweets parsing

In [ ]:
dates = [str(i)[:-9] + 'T00:00:00Z' for i in pd.date_range(start='11/01/2022', periods=150)]

master_ = pd.concat(master, axis=0)
pairs_arr = pd.DataFrame({'start':dates[:-1], 'end':dates[1:]})
pairs_arr.shape

In [ ]:
MY_BEARER_TOKEN = bt
client = tweepy.Client(bearer_token=MY_BEARER_TOKEN)

counter = 0

dfs = []

for search_query in tqdm(['balenciaga']):
    for i in range(pairs_arr.shape[0]):

        try:
            tweets = client.get_all_tweets_count(query=search_query,
                                                 start_time=pairs_arr['start'][i],
                                                 end_time=pairs_arr['end'][i],
                                                 granularity='minute'
                                                 )
            df = pd.DataFrame(tweets.data)
            print(search_query, df['tweet_count'].sum())

            df['search_query'] = search_query

            counter += df['tweet_count'].sum()

            dfs.append(df)

        except Exception as e:
            print(e)
            print('Waiting mode')
            time.sleep(900)

            tweets = client.get_all_tweets_count(query=search_query,
                                                 start_time=pair[0],
                                                 end_time=pair[1],
                                                 granularity='minute'
                                                 )
            df = pd.DataFrame(tweets.data)
            print(search_query, df['tweet_count'].sum())

            df['search_query'] = search_query

            counter += df['tweet_count'].sum()

            dfs.append(df)


dfs = pd.concat(dfs, axis=0)
dfs.to_pickle('time_ranges.pkl')

Creating df_all dataframe, that contains rows labeled with timeperiod based on tweet count thresholds.

- timeperiod = 1: Marks rows with cumulative tweet counts exceeding 450 or adjacent to rows exceeding the threshold.
- timeperiod = 2: Marks rows with isolated large jumps in cumulative counts.

In [ ]:
tags = ['balenciaga']

df_all = []

for tag in tags:
    df_local = dfs.copy().reset_index(drop=True)

    counter = 0
    counter2 = 0

    for i in tqdm(range(1, df_local.shape[0])):
        counter2 += df_local['tweet_count'][i]
        
        if df_local['tweet_count'][i] >= 450:
            if df_local['tweet_count'][i-1] < 450:
                df_local.at[i-1, 'timeperiod'] = 1
            df_local.at[i, 'timeperiod'] = 1
            counter = 0
            counter2 = 0
        elif counter2 >= 450 and counter < 450 and counter > 0:
            df_local.at[i-1, 'timeperiod'] = 1
            counter = df_local['tweet_count'][i]
            counter2 = df_local['tweet_count'][i]
        elif counter2 < 450 and counter < 450:
            counter += df_local.tweet_count[i]
        elif counter == 0 and counter2 > 450:
            df_local.at[i, 'timeperiod'] = 2
            counter = 0
            counter2 = 0
        
    df_all.append(df_local)
    
df_all = pd.concat(df_all, axis = 0)
df_all.reset_index(inplace=True, drop=True)

No rows have timeperiod = 2, indicating there were no isolated spikes or non-contiguous periods where cumulative tweet counts exceeded 450.

In [231]:
df_all[df_all['timeperiod'] == 2]

,end,start,tweet_count,search_query,timeperiod


Retrieves tweets within time periods scarped 

In [ ]:
MY_BEARER_TOKEN = bt
import time

def extract_geo(s):
    try:
        a = s['place_id']
        return a
    except:
        return None

df_arr_all = []
counter = 0

for i in tqdm(range(df_all.shape[0])):
    
    print(df_all['start'][i], df_all['end'][i])
    
    try:
        client = tweepy.Client(bearer_token=MY_BEARER_TOKEN)

        tweets = client.search_all_tweets(query=df_all['search_query'][i],
                                         start_time=df_all['start'][i],
                                         end_time=df_all['end'][i],
                                         tweet_fields = ["created_at", "text", "source", 'author_id', 'public_metrics',
                                                        'geo', 'entities', 'lang'],
                                         user_fields = ["name", "username", "location", "verified", "description"],
                                         place_fields = ['full_name', 'id', 'country', 'country_code', 'geo', 'name',
                                                        'place_type'],
                                         max_results = 500,
                                         expansions=['author_id', 'geo.place_id']
                                         )
    except:
        client = tweepy.Client(bearer_token=MY_BEARER_TOKEN)

        tweets = client.search_all_tweets(query=ch['search_query'][i],
                                         start_time=ch['start'][i],
                                         end_time=ch['end'][i],
                                         tweet_fields = ["created_at", "text", "source", 'author_id', 'public_metrics',
                                                        'geo', 'entities', 'lang'],
                                         user_fields = ["name", "username", "location", "verified", "description"],
                                         place_fields = ['full_name', 'id', 'country', 'country_code', 'geo', 'name',
                                                        'place_type'],
                                         max_results = 500,
                                         expansions=['author_id', 'geo.place_id']
                                         )
    counter += 1
    time.sleep(1)
    df_arr = []
    if counter == 290:
        time.sleep(900)
        counter = 0

    if tweets.meta['result_count'] > 0:

        print(f"Were found additionally {tweets.meta['result_count']} tweets")

        for el in tweets.data:
            df_arr.append([el.id, el.text, el.author_id, extract_geo(el.geo), el.created_at, 
                           el.lang, el.public_metrics.get('retweet_count', 0), el.public_metrics.get('reply_count', 0),
                           el.public_metrics.get('like_count', 0), el.public_metrics.get('quote_count', 0)])

        df = pd.DataFrame(df_arr)

        df.columns = ['tweet_id', 'text', 'author_id', 'geo', 'created_at', 'lang', 'retweet_count', 'reply_count',
                     'like_count', 'quote_count']

        df['author_location'] = '-1'
        df['country'] = 'No Country
        df['query'] = df_all['search_query'][i]

        df_arr_all.append(df)

pd.concat(df_arr_all, axis=0).to_pickle('balenciaga_tweets.pkl')
dff = pd.concat(df_arr_all, axis=0)

Retriving user's information

In [262]:
users = dff['author_id'].unique()
df = pd.DataFrame({'author_id':users})
users_prev = df_prev['author_id'].unique()
df = df[~df['author_id'].isin(users_prev)].reset_index(drop=True)
df

,author_id
0,1123166102995865600
1,1950852385
2,50073553
3,1388702657792159746
4,49987009
...,...
58067,198146259
58068,704059993
58069,1526758613775155200
58070,1235772718043316226


In [ ]:
headers = {
    'Authorization': f"Bearer {bt}",
}

profile_arr = []
counter = 0

for i in tqdm(range(df.shape[0] // 100 + 1)):
    if counter == 295:
        time.sleep(900)
        counter = 0
        
    author_ids = ','.join([str(i) for i in df.iloc[i * 100:(i+1)*100, :]['author_id'].tolist()])
    s = f"https://api.twitter.com/2/users?ids={author_ids}&user.fields=location,name"
    response = requests.get(s, headers=headers)
    r = json.loads(response.text)['data']
    profile_arr.append(pd.DataFrame(r))
    counter += 1

In [264]:
df = pd.concat(profile_arr, axis = 0).reset_index(drop=True)
df = df.drop(columns = ['withheld']).drop_duplicates().reset_index(drop=True)
df['loc1'] = df['location'].str.lower().fillna('no country')
df['country'] = 'no country'
df['id'] = df['id'].apply(int)
df.to_pickle('users_balenciaga.pkl')

Merging user's information and tweets

In [ ]:
dff2 = dff.merge(df.rename(columns = {"id":'author_id'}), how = 'left', on = 'author_id')
dff2.to_pickle('balenciaga.pkl')